In [2]:
import numpy as np
import pandas as pd
##_____________________________

# make_data_set 함수: csv 파일을 numpy로 반환, 추가로 csv 파일의 열 이름도 반환
def make_data_set(num_features, act, trial, sub_id):
    data_set = np.zeros((0, num_features))
    fname = 'A_DeviceMotion_data/' + act + '_' + str(trial) + '/sub_' + str(sub_id)+'.csv'
    raw_data = pd.read_csv(fname)
    raw_data = raw_data.drop(['Unnamed: 0'], axis=1)
    unlabel_data = raw_data.values
    data_set = np.append(data_set, unlabel_data, axis = 0)
    pd.DataFrame(data_set).to_csv('A_DeviceMotion_data/' + act + '_' + str(trial) + '/sub_' + str(sub_id)+'_origin.csv', header=raw_data.columns, index=1)
    return data_set, list(raw_data.columns)
#________________________________

# bdi_comp 함수: BDI로 압축한 numpy를 반환
def bdi_comp(before_bdi):
    ## float 형식의 변수를 int로 바꾸기 위해 1000000을 곱해준다
    bdi = before_bdi * 1000000
    bdi = bdi.astype(int)

    ## 각 열의 최대값과 최소값의 평균을 base로 설정하고, immediate 계산
    base_list = []
    for i in range(bdi.shape[1]):
        base = round((np.max(bdi[:, i]) + np.min(bdi[:, i])) / 2)
        base_list.append(base)
        for j in range(bdi.shape[0]):
            bdi[j, i] = bdi[j, i] - base

    ## 마지막 행에 각 열의 base 값을 추가
    bdi = np.append(bdi, np.array(base_list).reshape(1, -1), axis=0)
    return bdi
#________________________________

print("--> Start...")
## 불러올 csv 파일의 이름 및 위치를 정한다
num_features = 12 # attitude(roll, pitch, yaw); gravity(x, y, z); rotationRate(x, y, z); userAcceleration(x,y,z)
act = "dws" # dws, ups, wlk, jog, sit, std
trial = 1 # 0 ~ 16
sub_id = 1 # 1 ~ 24

## make_data_set으로 csv 파일을 ds 변수에 저장
print("--> CSV to Numpy data set")
ds, columns_list = make_data_set(num_features, act, trial, sub_id)
print("--> Numpy data set shape:", ds.shape)
print(ds)

## bdi_comp로 BDI 압축 실행
print("--> BDI Compression Start")
bdi_ds = bdi_comp(ds)
print("--> Numpy data set shape:", bdi_ds.shape)
print(bdi_ds)

## numpy를 csv 파일로 저장
print("--> Numpy to CSV")
pd.DataFrame(bdi_ds).to_csv('A_DeviceMotion_data/' + act + '_' + str(trial) + '/sub_' + str(sub_id)+'_comp.csv', header=columns_list, index=1)

--> Start...
--> CSV to Numpy data set
--> Numpy data set shape: (1751, 12)
[[ 1.528132 -0.733896  0.696372 ...  0.294894 -0.184493  0.377542]
 [ 1.527992 -0.716987  0.677762 ...  0.219405  0.035846  0.114866]
 [ 1.527765 -0.706999  0.670951 ...  0.010714  0.134701 -0.167808]
 ...
 [ 1.830821 -0.578367  2.447967 ... -0.08559  -0.030209 -0.08774 ]
 [ 1.849557 -0.586962  2.439458 ... -0.048105  0.029555  0.060441]
 [ 1.869375 -0.596783  2.433775 ... -0.065011 -0.042575  0.046052]]
--> BDI Compression Start
--> Numpy data set shape: (1752, 12)
[[ 134927  -42927  693436 ...  113816 -591585  458060]
 [ 134787  -26018  674826 ...   38327 -371247  195384]
 [ 134560  -16029  668015 ... -170364 -272391  -87289]
 ...
 [ 456352  104007 2436522 ... -229183 -377537  140958]
 [ 476170   94186 2430839 ... -246089 -449667  126569]
 [1393205 -690969    2936 ...  181078  407092  -80518]]
--> Numpy to CSV
